In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [ ]:
def build_cnn(input_shape):
    model=Sequential()
    activation = mish
    ks = 3
    strides = 2
    ps = 2
    # Layer 1
    model.add(Conv1D (kernel_size=ks, filters=128, strides=strides, padding='valid',
                      kernel_initializer='he_uniform', input_shape=input_shape))                  
    model.add(BatchNormalization())
    model.add(Activation(activation))

    # Layer 2
    model.add(Conv1D (kernel_size=ks, filters=128, padding='same', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling1D(pool_size=ps, strides=strides))

    # Layer 3
    model.add(Conv1D (kernel_size=ks, filters=256, padding='same', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling1D(pool_size=ps, strides=strides))

    # Layer 4
    model.add(Conv1D (kernel_size=ks, filters=256, padding='same', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(MaxPooling1D(pool_size=ps, strides=strides))

    model.add(GlobalMaxPooling1D())
    model.add(BatchNormalization())
    
    model.add(Dense(256, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    
    model.add(Dense(128, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.5))


    # Layer 12
    model.add(Dense(30))
    model.add(Activation('softmax'))
    return model

In [ ]:
# Check List
## data shape: (n, ts_interval, n_cols)
## output type: clf, reconstruction
model = build_cnn((32, 1024))

In [ ]:
model.summary()